In [3]:
import json
import requests
import re
from pprint import pprint
import pandas as pd

The `.env` file is with the structure:
```
aol_key=******
aol_username=username
cc_api_key=********
cc_token=*********
```

In [9]:
env_path = ".env-sample"

In [11]:
tmp = []
with open(env_path) as f:
    ### Since the order of the env might change, it might be better to pull by the 'key' instead of line
    
    # Create an empty env object
    env = {}
    for line in f:
        env_key, _val = line.split("=")
        env_value = _val.split("\n")[0] 
        
        env[env_key] = env_value
        
#     ### Or, you could do this in one line using a "Dictionary Comprehension"
#     env = {
#          line.split("=")[1]: line.split("=")[1].split("\n")[0] 
#                for line in f }
        
        


api_key = env['cc_api_key']
token = env['cc_token']

pprint(env)

{'aol_key': 'this_is_a_key',
 'aol_username': 'username',
 'cc_api_key': 'this_also_is_a_key',
 'cc_token': 'this_is_a_token'}


To get the token put this in the web browser: https://api.cc.email/v3/idfed?client_id={api_key}&redirect_uri=https://localhost&response_type=token&scope=contact_data

In [12]:
headers = {
  'Authorization': f'Bearer {token}'
}
payload = {}

In [16]:
url_contact_lists = "https://api.cc.email/v3/contact_lists?include_count=false"

try:
    ## Create empty object first
    r_json = {}
    
    ## cleaner way of using requests
    response = requests.get(url_contact_lists, headers=headers, data=payload)

except:
    print("There was an problem in the request :(")
    
    
## always nice to print the url as a sanity check
print(response.url)

# if succesful, populate your response json
if  response.status_code == 200:
        r_json = response.json()
else:
    print(f'Failed to get tile {response.status_code}, {response.json()}')
    

In [ ]:
# Or, as a reusable function
def executeRequest(url, headers, payload):
    
    try:
    ## Create empty object first
    r_json = {}
    
    ## cleaner way of using requests
    response = requests.get(url, headers=headers, data=payload)

    except:
        print("There was an problem in the request :(")
        return None

    ## always nice to print the url as a sanity check
    print(response.url)

    # if succesful, populate your response json
    if  response.status_code == 200:
            r_json = response.json()
    else:
        print(f'Failed to get tile {response.status_code}, {response.json()}')
    
    return r_json


At this point the response is a dictionary with a unique element `lists`

In [38]:
## Always good to have a fallback
l_json = r_json.get("lists", [])

We get the elements inside lists, and what we get is a list of dictionaries

In [22]:
# Can use asserts to make sure we get what we need to continue
# This is useful when you wrap this in functions

assert type(l_json) is list and len(l_json) is not 0,  "Error with l_json"

At this point we go through the list of dictionaries to find those contact_lists that have in the name `Educator Ambassadors`.

In [67]:
# sel_contact_dict = {}
# for d in l_json:
#     m = re.search('Educator Ambassadors', d['name'])
#     if m != None:
#         sel_contact_dict[d['name']] = d['list_id']
        
        
# Again, here we can do this in 'one' line

sel_contact_dict = {
    d['name']: d.get('list_id', '')
    for d in l_json
    if re.search('Educator Ambassadors', d['name']) != None
}

In [73]:
len(sel_contact_dict)

6

There are 6 lists that contain `Educator Ambassadors` in the name. We restrictthe matching of the name to those that exactly have the name `Educator Ambassadors`. 

In [69]:
# sel_contact_lists = []
# for d in l_json:
#     m = re.search('^Educator Ambassadors$', d['name'])
#     if m != None:
#         sel_contact_lists.append(d['list_id'])


sel_contact_list = [
    d['list_id']
    for d in l_json
    if re.search('^Educator Ambassadors$', d['name']) != None
]

In [75]:
id_contact_list = sel_contact_lists[0]

In [76]:
# here you could use executeRequest

url_contacts = f"https://api.cc.email/v3/contacts?lists={id_contact_list}&include=street_addresses&limit=500&include_count=false"

In [77]:
headers = {
  'Authorization': f'Bearer {token}'
}
payload = {}

In [78]:
response = requests.request("GET", url_contacts, headers=headers, data = payload)

In [79]:
r_json = response.json()

In [82]:
l_json = r_json["contacts"]

In [106]:
len(l_json)

456

In [152]:
contacts_list = []
for d in l_json:
#     if d['street_addresses']:        
#         if d['street_addresses'][0].get("postal_code") != None and d['street_addresses'][0].get("country") != None:
#             contact_dict = {
#                 'contact_id':d['contact_id'],
#                 'postal_code':d['street_addresses'][0]['postal_code'],
#                 'country':d['street_addresses'][0]['country']
#             }
#             contacts_list.append(contact_dict)
            

    # You can make teh logic more concise using the .get fallbacks the 
    street_address = d.get('street_addresses', [{}])[0]
    
    postal_code = street_address.get("postal_code", None)
    country = street_address.get("country", None)
    
    
    if postal_code and country:
            contact_dict = {
                'contact_id': d.get('contact_id', ''), ## Need a fallback for contact_id?
                'postal_code': postal_code,
                'country': country
            }
            contacts_list.append(contact_dict)
            

In [149]:
len(contacts_list)

443

In [155]:
df = pd.DataFrame(contacts_list)

In [167]:
## Good practise to use relative path
csv_file = './out.csv'

In [157]:
df.to_csv(csv_file, index=False)

# Once the csv is ready it can be published in arcgis online via the arcgis api

In [158]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

In [165]:
tmp = []
with open(".env") as f:
    for line in f:
        tmp.append(line)
aol_password = tmp[0].split("=")[1].split("\n")[0]
aol_username = tmp[1].split("=")[1].split("\n")[0]

In [166]:
#getting into the GIS
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

In [ ]:
csv_item = gis.content.add({}, csv_file)
csv_lyr = csv_item.publish(None,  { 'CountryCode' : 'country',
                                    'Postal' : 'postal_code'} )

In [ ]:
new_folder_details = gis.content.create_folder("constant_contact")
csv_item.move(new_folder_details)
csv_lyr.move(new_folder_details)
csv_lyr.share(everyone = True)
#print(csv_lyr.ownerFolder)